In [ ]:
from functools import partial
from collections import namedtuple
import random

from IPython import display
import ipywidgets
from ipywidgets import Button, Text, SelectMultiple, HBox, VBox,\
                       Textarea, Image, Label, HTML, Dropdown,\
                       IntProgress, Tab, IntText, Select, Checkbox
import traitlets
import pandas
import jinja2

In [ ]:
from app_model import AppModel

state = AppModel()

In [ ]:
### Javascript kludges to add autocompletion to text boxes and enable multiple selections

autocomplete = jinja2.Template("""
  $( function() {
    var availableTags = [
    {%- for option in options %}
        "{{ option }}",
    {%- endfor %}
    ];
    $( ":input[{{ selector }}]" ).autocomplete({
      minLength: 2,
      source: availableTags
    });
  } );
""")

prevent_unselect = """
window.onmousedown = function (e) {
    var el = e.target;
    if (el.tagName.toLowerCase() == 'option' && el.parentNode.hasAttribute('multiple')) {
        e.preventDefault();

        // toggle selection
        if (el.hasAttribute('selected')) el.removeAttribute('selected');
        else el.setAttribute('selected', '');

        // hack to correct buggy behavior
        var select = el.parentNode.cloneNode(true);
        el.parentNode.parentNode.replaceChild(select, el.parentNode);
    }
}
"""

# Widget definitions and interaction

In [ ]:
## Solution selection (submission x problem) matrix
prev_stud = Button(icon='fa-step-backward', description=' previous')
next_stud = Button(icon='fa-step-forward', description=' next')
next_ungraded_stud = Button(icon='fa-forward', description=' next ungraded')
jump_to_stud = Text(description='Jump to', placeholder='student # or name')

jump_to_exam = Dropdown(description='Exam', placeholder='exam #')

traitlets.link((state, 'exams'), (jump_to_exam, 'options'))
jump_to_exam.value = state.id_to_exam()
traitlets.directional_link((jump_to_exam, 'value'), (state, 'exam_id'), state.exam_to_id)

jump_to_problem = Dropdown(description='Jump to', placeholder='problem #')

traitlets.link((state, 'problems'), (jump_to_problem, 'options'))
traitlets.directional_link((jump_to_problem, 'value'), (state, 'problem_id'), state.problem_to_id)

## Solution and FeedbackOption
options_feedback = SelectMultiple(name='feedback')
traitlets.directional_link((state, 'feedback_options'), (options_feedback, 'options'))

remarks = Textarea(placeholder='Student-specific feedback')
traitlets.link((state, 'remarks'), (remarks, 'value'))

full_page = Checkbox(description='Show full page')
traitlets.link((state, 'show_full_page'), (full_page, 'value'))

answer = Image(format='png', width='100%')

## info
graded_label = HTML()
graded_label.template = '<h5>Graded by: {} @ {:%d/%m %H:%M}</h5>'.format

problem_progress = IntProgress(
    value=0,
    min=0,
    step=1,
    description='Progress:',
    bar_style='success',
    orientation='horizontal',
)

traitlets.directional_link((state, 'exam_id'), (problem_progress, 'max'), state.num_submissions)

## grader selection
grader = Dropdown(description='Grader', placeholder='Grader name', options=state.graders)
widgets_to_disable = (options_feedback, remarks)
for w in widgets_to_disable:
    traitlets.directional_link((state, 'grader_id'), (w, 'disabled'), transform=lambda x: x == 0)

# 
option_names = Select(description='Feedback options', options=['ADD NEW'])

traitlets.directional_link((state, 'feedback_options'), (option_names, 'options'),
                           transform=lambda x: ['ADD NEW'] + x)

traitlets.link((option_names, 'value'), (state, 'edited_feedback_option'))

traitlets.directional_link((option_names, 'value'), (state, 'edited_feedback_name'),
                           lambda x: '' if x == 'ADD NEW' else x)

option_name = Text(placeholder='Option name')
traitlets.link((state, 'edited_feedback_name'), (option_name, 'value'))

option_score = IntText(description='Score')
traitlets.link((option_score, 'value'), (state, 'edited_score'))

option_description = Textarea(placeholder='Full description', width='100%')
traitlets.link((option_description, 'value'), (state, 'edited_description'))

save_option = Button(description='Update/add', button_style='success')
delete_option = Button(description='Delete', button_style='danger')

def render_ui(_=None):
    image, feedback, grader = state.get_solution()
    answer.value = image
    jump_to_stud.value = state.student
    jump_to_problem.value = state.problem()
    render_feedback(*feedback)
    render_grader(*grader)

def render_feedback(options, selected, _remarks):
    options_feedback.value = selected

def render_grader(graded_by, graded_at):
    if (graded_by is None) ^ (graded_at is None):
        raise ValueError('Invalid grading state')
    if graded_by is None:
        graded_label.value = ''
    else:
        graded_label.value = graded_label.template(graded_by, graded_at)
    problem_progress.value = state.num_graded()


state.observe(render_ui, names=['exam_id', 'submission_id', 'problem_id', 
                                'feedback_options', 'show_full_page'])

## Utilities

def observe(widget, **kwargs):
    return partial(widget.observe, **kwargs)

def load_image(path):
    with open(path, 'rb') as f:
        return f.read()    

## actual controllers - abusing decorator syntax FTW!

prev_stud.on_click(lambda _: state.previous_submission())
next_stud.on_click(lambda _: state.next_submission())
next_ungraded_stud.on_click(lambda _: state.next_ungraded())

@jump_to_stud.on_submit
def _(sender):
    try:
        student_id = int(sender.value.strip()[:7])
    except ValueError:
        return
    state.jump_to_student(student_id)
    
@observe(grader, names="value")
def _(sender):
    if sender['new'] == 'None':
        state.set_grader(None)
    else:   
        state.set_grader(sender['new'])

traitlets.link((options_feedback, 'value'), (state, 'selected_feedback'))
traitlets.link((remarks, 'value'), (state, 'remarks'))

## Controllers for the feedback options editing

save_option.on_click(state.commit_feedback_edit)
delete_option.on_click(state.delete_feedback_option)

# Layout

In [ ]:
render_ui()

In [ ]:
HBox([grader, jump_to_exam, jump_to_problem])

In [ ]:
# Main layout

grade = VBox([HBox([prev_stud, next_stud, next_ungraded_stud, jump_to_stud]),
              HBox([options_feedback, remarks]),
              HBox([problem_progress,  graded_label, full_page]), 
              answer])

edit_feedback = VBox([option_names, 
                      HBox([option_name, 
                            option_score,
                            save_option,
                            delete_option]),
                      
                      option_description])

editing = Tab(children=[grade, edit_feedback])
editing.set_title(0, 'Grade')
editing.set_title(1, 'Edit feedback options')
editing

In [ ]:
# apply javascript kludges
# must trigger after the widgets were rendered

selector = "placeholder='{}'".format(jump_to_stud.placeholder)
display.display_javascript(display.Javascript(autocomplete.render(selector=selector, options=state.students)))
display.display_javascript(prevent_unselect)